In [ ]:
# UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
# Emelyn Clementino Freire - 20220043970
# Pablo Durkheim Fernandes do Nascimento - 20230000715
# ----------- Tarefa Gerenciamento de Threads ------------------

import threading
from PIL import Image
import numpy as np

# Função para calcular Gx (detecção de bordas na direção x)
def calculate_Gx(image, result, start_row, end_row):
    try:
        for i in range(start_row, end_row):
            for j in range(1, image.shape[1] - 1):
                Gx = (image[i + 1, j - 1] + 2 * image[i + 1, j] + image[i + 1, j + 1]) - \
                     (image[i - 1, j - 1] + 2 * image[i - 1, j] + image[i - 1, j + 1])
                result[i, j] = max(0, min(255, Gx))
    except IndexError:
        pass

# Função para calcular Gy (detecção de bordas na direção y)
def calculate_Gy(image, result, start_row, end_row):
    try:
        for i in range(start_row, end_row):
            for j in range(1, image.shape[1] - 1):
                Gy = (image[i - 1, j + 1] + 2 * image[i, j + 1] + image[i + 1, j + 1]) - \
                     (image[i - 1, j - 1] + 2 * image[i, j - 1] + image[i + 1, j - 1])
                result[i, j] = max(0, min(255, Gy))
    except IndexError:
        pass

# Função principal da thread mãe
def main_thread():
    # Abre a imagem em nível de cinza
    img = Image.open("imgs/coins.png")
    image_array = np.array(img)

    M, N = image_array.shape

    # Inicializa as matrizes Gx, Gy e G
    Gx = np.zeros((M, N), dtype=np.uint8)
    Gy = np.zeros((M, N), dtype=np.uint8)
    G = np.zeros((M, N), dtype=np.uint8)

    # Divide o trabalho entre as threads para Gx e Gy
    mid_row = M // 2
    thread1 = threading.Thread(target=calculate_Gx, args=(image_array, Gx, 1, mid_row))
    thread2 = threading.Thread(target=calculate_Gy, args=(image_array, Gy, mid_row, M))

    # Inicia as threads
    thread1.start()
    thread2.start()

    # Aguarda as threads terminarem
    thread1.join()
    thread2.join()

    # Combina as imagens Gx e Gy para obter a imagem de saída G
    G = Gx + Gy
    G[G > 255] = 255

    # Crie uma imagem PIL a partir da matriz G
    output_image = Image.fromarray(G)

    # Salva a imagem de saída
    output_image.save("imgs/imagemDeSaida.png")

if __name__ == "__main__":
    main_thread()